In [1]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests
import math

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)

from datetime import datetime, timedelta

global conn
global cursor 


def read_automode_24hrs_before(relayNumber):
    conn = sqlite3.connect('../database/pythonsqlite.db')
    cursor = conn.cursor()
    
    current_datetime = datetime.now() - timedelta(hours=24)
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
    cursor.execute(query, (relayNumber,))
    
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)

    if int(result_list[0][0]) == 0: # if no results then check 48 hrs before data

        current_datetime = datetime.now() - timedelta(hours=48)
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

        query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
        cursor.execute(query, (relayNumber,))
        
        rows = cursor.fetchall()
        result_list = []
        for row in rows:
            result_list.append(row)


        cursor.close()

        return result_list
    else:
        cursor.close()
        return result_list


def getAutoModeStatus():
    try:
        current_datetime = datetime.now()
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)
        # Format datetime with 'T'
        start_of_day_str = start_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
        end_of_day_str = end_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]

        # Connect to the SQLite database
        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")

        df = pd.read_sql_query(f"SELECT start_timestamp, end_timestamp, marketprice, unit, relaynumber FROM automaterelay WHERE start_timestamp BETWEEN '{start_of_day_str}' AND '{end_of_day_str}'", conn)

        # Convert DataFrame to JSON
        json_data = df.to_json(orient='records')

        print(df)

        # Return JSON response
        return json_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"status": "Error"}), 500

    finally:
          if 'conn' in locals() and conn:
            conn.close()


def calculateAutoModeValue(relayNumber):
    db = DatabaseManager()

    if(relayNumber == 1):
        relayPower = db.read_relaysettings_table()[0][2] # relay 1 power
    elif(relayNumber == 2):
        relayPower = db.read_relaysettings_table()[0][3] # relay 2 power
    
    last_known_times_toturn_on = db.read_automode_24hrs_before(relayNumber)
    last_known_times_toturn_on = int(last_known_times_toturn_on[0][0]) * int(relayPower)
    last_24hrs_usage = db.read_datacache_last_24hrs_consumption()[0][0] # A
    
    """
    Formula to calculate auto_mode value
    A = Last 24hrs consumption
    B = Power needed for Relay1 or Relay 2
    Result = A/B

    Example: A = 20KW; B = 5KW
        Result = 20/5 
                = 4 times (this we have to turn on and turn off the relay automatically)

    Escalation Example:
        Lets say times_turnon=6*Relay power (20kwh)
            E old = 6*20 = 120 kwh 
        If the last 24hrs consumption is greater or equal to (E Old) then multiply with escalation value
    """
    
    # Logic for Escalation --> 1.3 default value
    # Higher the demand higher the turn on time
    if(last_known_times_toturn_on >= last_24hrs_usage):
        no_of_times_to_activate_automode = last_24hrs_usage * 1.3 / int(relayPower)
        db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 
    else:
        no_of_times_to_activate_automode = int(last_24hrs_usage) / int(relayPower)
        db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 

    return True



def getAutoModeStatus():
    try:
        current_datetime = datetime.now()
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)
        # Format datetime with 'T'
        start_of_day_str = start_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
        end_of_day_str = end_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]

        # Connect to the SQLite database
        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")

        query = f"SELECT REPLACE(start_timestamp, 'T', ' ') AS start_timestamp, REPLACE(end_timestamp, 'T', ' ') AS end_timestamp, marketprice, unit, relaynumber FROM automaterelay WHERE (start_timestamp >= '{start_of_day_str}' AND start_timestamp <= '{end_of_day_str}') OR (end_of_day_str >= '{start_of_day_str}' AND end_of_day_str <= '{end_of_day_str}')"

        df = pd.read_sql_query(query, conn)

        # Convert DataFrame to JSON
        json_data = df.to_json(orient='records')

        # Return JSON response
        return json_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"status": "Error"}), 500

    finally:
          if 'conn' in locals() and conn:
            conn.close()



#getAutoModeStatus()

def read_automode_24hrs_before(relayNumber):
    conn = sqlite3.connect('../database/pythonsqlite.db')
    cursor = conn.cursor()
    
    current_datetime = datetime.now() - timedelta(hours=24)
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
    cursor.execute(query, (relayNumber,))
    
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)


    if int(result_list[0][0]) == 0: # if no results then check 48 hrs before data

        current_datetime = datetime.now() - timedelta(hours=48)
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

        query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
        cursor.execute(query)
        
        rows = cursor.fetchall()
        result_list = []
        for row in rows:
            result_list.append(row)

        cursor.close()
        return result_list
    else:
        cursor.close()
        return result_list


def auto_mode(relayNumber):
    db = DatabaseManager()
    times_toturn_on = db.read_automode_24hrs_before(relayNumber)
    if not times_toturn_on:
        return
    else:
        times_toturn_on = int(times_toturn_on[0][0])
    
    print("No of times to turn on the relay", times_toturn_on)
    future_df = AwattarServices().AWATTAR_FUTURE_PRICE()
    print("Data from BEFORE modification", future_df)

    date_time = datetime.now()
    print(date_time)

    # delete past values from future_df
    # self.future_df = self.future_df[self.future_df['start_timestamp'] >= self.date_time.now()]

    # find min 2 values of future_df
    future_df.sort_values(by=['marketprice'], inplace=True)
    future_df = future_df.iloc[:times_toturn_on]
    future_df.sort_values(by=['start_timestamp'], inplace=True)
    # add extra columns to data frame and to match database table "automaterelay"
    future_df['triggerstatus'] = True
    future_df['unit'] = "kWh"
    future_df['relaynumber'] = relayNumber

    print("Data from AFTER modification", future_df)

    if len(future_df) > 0:
        print("Found matching auto mode for relay:", relayNumber)
        #conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
        #self.future_df.to_sql('automaterelay', conn, index=False, if_exists='append') # replace the dataset
        #self.future_df.to_sql('automaterelay_report', conn, index=False, if_exists='append') # for report purpose
        #conn.close()
    else:
        print("No matching auto mode")

    #self.turn_on_turn_off(relayNumber)



def calculateAutoModeValue(relayNumber):
    db = DatabaseManager()

    if(relayNumber == 1):
        relayPower = db.read_relaysettings_table()[0][2] # relay 1 power
    elif(relayNumber == 2):
        relayPower = db.read_relaysettings_table()[0][3] # relay 2 power
    
    last_known_times_toturn_on = db.read_automode_24hrs_before(relayNumber)
    last_known_times_toturn_on = int(last_known_times_toturn_on[0][0]) * int(relayPower)
    last_24hrs_usage = db.read_datacache_last_24hrs_consumption()[0][0] # A
    
    """
    Formula to calculate auto_mode value
    A = Last 24hrs consumption
    B = Power needed for Relay1 or Relay 2
    Result = A/B

    Example: A = 20KW; B = 5KW
        Result = 20/5 
                = 4 times (this we have to turn on and turn off the relay automatically)

    Escalation Example:
        Lets say times_turnon=6*Relay power (20kwh)
            E old = 6*20 = 120 kwh 
        If the last 24hrs consumption is greater or equal to (E Old) then multiply with escalation value
    """
    
    # Logic for Escalation --> 1.3 default value
    # Higher the demand higher the turn on time
    if(last_known_times_toturn_on >= last_24hrs_usage):
        no_of_times_to_activate_automode = last_24hrs_usage * 1.3 / int(relayPower)
        ## db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 
        db.insert_automode(last_24hrs_usage, relayNumber,  math.ceil(no_of_times_to_activate_automode)) 

    else:
        no_of_times_to_activate_automode = int(last_24hrs_usage) / int(relayPower)
        ## db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 
        db.insert_automode(last_24hrs_usage, relayNumber,  math.ceil(no_of_times_to_activate_automode)) 

calculateAutoModeValue(2)


from datetime import datetime

current_datetime = datetime.now()
print("Current Date and Time:", current_datetime)

unixdt = current_datetime.timestamp()
print(unixdt)

 
# Start of the day
start_of_day = datetime(current_datetime.year, current_datetime.month, current_datetime.day, 0, 0, 0, 0, tzinfo=timezone.utc)

# End of the day
end_of_day = start_of_day + timedelta(days=1) - timedelta(microseconds=1)

print(start_of_day)
print(end_of_day)




KeyboardInterrupt: 

In [ ]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt
import pytz
from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)



In [ ]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt
import pytz
from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

def calculateAutoModeValue(relayNumber):
        db = DatabaseManager()

        if(relayNumber == 1):
            relayPower = db.read_relaysettings_table()[0][2] # relay 1 power
        elif(relayNumber == 2):
            relayPower = db.read_relaysettings_table()[0][3] # relay 2 power
        
        last_known_times_toturn_on = db.read_automode_24hrs_before(relayNumber)
        last_known_times_toturn_on = int(last_known_times_toturn_on[0][0]) * int(relayPower)
        last_24hrs_usage = db.read_datacache_last_24hrs_consumption()[0][0] # A
        
        """
        Formula to calculate auto_mode value
        A = Last 24hrs consumption
        B = Power needed for Relay1 or Relay 2
        Result = A/B

        Example: A = 20KW; B = 5KW
            Result = 20/5 
                   = 4 times (this we have to turn on and turn off the relay automatically)

        Escalation Example:
            Lets say times_turnon=6*Relay power (20kwh)
                E old = 6*20 = 120 kwh 
            If the last 24hrs consumption is greater or equal to (E Old) then multiply with escalation value
        """
      
        # Logic for Escalation --> 1.3 default value
        # Higher the demand higher the turn on time
        if(last_known_times_toturn_on >= last_24hrs_usage):
            no_of_times_to_activate_automode = last_24hrs_usage * 1.3 / int(relayPower)
            print(round(no_of_times_to_activate_automode))
            ## db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 
        else:
            no_of_times_to_activate_automode = int(last_24hrs_usage) / int(relayPower)
            ## db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 
            print(round(no_of_times_to_activate_automode))

        return True

## calculateAutoModeValue(1)


def turn_on_turn_off(relayNumber):
        # check whether the relay 1 or 2 is on Auto mode then turn on and turn off automatically
        db = DatabaseManager()
        results = db.read_relaymode_temp(relayNumber) # setting relay number
                
        current_datetime = datetime.now() 
        current_datetime = current_datetime.strftime('%Y-%m-%dT%H:00:00')

        buffer_datetime = datetime.now() + timedelta(hours=1)
        buffer_datetime = buffer_datetime.strftime('%Y-%m-%dT%H:00:00')

        print(current_datetime)
        print(buffer_datetime)

        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
        query = f"SELECT * from automaterelay where relaynumber={relayNumber} and start_timestamp BETWEEN '{current_datetime}' and '{buffer_datetime}'"
       
       # query = f"SELECT * from automaterelay where relaynumber={relayNumber} and start_timestamp BETWEEN '2023-12-07T22:00:00' and '2023-12-07T23:00:00'"
        print(query)
        
        new_dataframe = pd.read_sql_query(query, conn)

        print(new_dataframe)


        if len(results) > 0: # check whether relay is on Auto Mode
                # check if new_dataframe is empty then turn off the relay
                if(new_dataframe.empty):
                     # Turn OFF
                    print(f"Relay {relayNumber} turned off in AutoMode")

                current_time = datetime.now()
                # Check if current time is within any interval
                for index, row in new_dataframe.iterrows():
                    startdatetime = pd.to_datetime(row['start_timestamp'])
                    enddatetime = pd.to_datetime(row['end_timestamp'])
                    print("startdatetime", startdatetime)
                    print("enddatetime", enddatetime)
                    print("current_time", current_time)

                    if startdatetime <= current_time <= enddatetime:
                        # Turn On
                        print(f"Relay {relayNumber} turned on in AutoMode")
                        break
                    else:
                        # Turn OFF
                        print(f"Relay {relayNumber} turned off in AutoMode")


turn_on_turn_off(1)


 
def read_automode_24hrs_before(relayNumber):
    conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
    cursor = conn.cursor()

    current_datetime = datetime.now() - timedelta(hours=24)
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
    cursor.execute(query, (relayNumber,))
    
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)


    if int(result_list[0][0]) == 0: # if no results then check 48 hrs before data

        current_datetime = datetime.now() - timedelta(hours=48)
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

        query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
        cursor.execute(query, (relayNumber,))

        
        rows = cursor.fetchall()
        result_list = []
        for row in rows:
            result_list.append(row)

        cursor.close()
        return result_list
    else:
        cursor.close()
        return result_list

    
def read_automode_48hrs_before(relayNumber):
    conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
    cursor = conn.cursor()
    
    current_datetime = datetime.now() - timedelta(hours=48)
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    query = f"SELECT times_to_turnon FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}';"
    cursor.execute(query, (relayNumber,))
    
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)
    
    cursor.close()
    return result_list

## read_automode_24hrs_before(1)



def read_datacache_last_24hrs_consumption():
    current_datetime = datetime.now() - timedelta(hours=24)
    print(current_datetime.date())
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
    cursor = conn.cursor()
    query = f"SELECT COALESCE(SUM(smart_meter_consumption), 0) AS consumption, smart_meter_unit as unit FROM datacache_report WHERE start_timestamp BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 25"
    cursor.execute(query)
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)


    if int(result_list[0][0]) == 0: # if no results then check 48 hrs before data

        current_datetime = datetime.now() - timedelta(hours=48)
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

        query = f"SELECT COALESCE(SUM(smart_meter_consumption), 0) AS consumption, smart_meter_unit as unit FROM datacache_report WHERE start_timestamp BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 25"
        cursor.execute(query)
        
        rows = cursor.fetchall()
        result_list = []
        for row in rows:
            result_list.append(row)

        cursor.close()
        return result_list
    else:
        cursor.close()
        return result_list


def read_datacache_last_48hrs_consumption():
    current_datetime = datetime.now() - timedelta(hours=48)
    print(current_datetime.date())
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
    cursor = conn.cursor()
    cursor.execute(
        f"SELECT sum(smart_meter_consumption) as consumption, smart_meter_unit as unit FROM datacache WHERE start_timestamp BETWEEN '{start_of_day}' AND '{end_of_day}'")
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)
    cursor.close()
    return result_list

## read_datacache_last_24hrs_consumption()

  
from datetime import datetime, timedelta
import pytz

def get_unix_timestamp_vienna(date, start_of_day=True):
    # Set the time zone for Vienna
    vienna_timezone = pytz.timezone('Europe/Vienna')

    # Set the time to midnight for the specified date
    vienna_date = vienna_timezone.localize(datetime.combine(date, datetime.min.time()))

    # Set the time to the start or end of the day
    if start_of_day:
        vienna_date = vienna_date.replace(hour=0, minute=0, second=0, microsecond=0)
    else:
        vienna_date = vienna_date.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Convert the datetime to Unix timestamp
    unix_timestamp = int((vienna_date - datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds())

    return unix_timestamp

# Example: Get Unix timestamp for the start and end of today in Vienna
today = datetime.now().date()
start_of_day_timestamp = get_unix_timestamp_vienna(today, start_of_day=True)
end_of_day_timestamp = get_unix_timestamp_vienna(today, start_of_day=False)

print(f"Start of the day in Vienna: {start_of_day_timestamp}")
print(f"End of the day in Vienna: {end_of_day_timestamp}")

def unix_timestamp_to_datetime(unix_timestamp, timezone='UTC'):
    # Convert Unix timestamp to datetime object
    dt = datetime.utcfromtimestamp(unix_timestamp)

    # Set the timezone
    dt = dt.replace(tzinfo=pytz.timezone(timezone))

    return dt

print(unix_timestamp_to_datetime(start_of_day_timestamp))
print(unix_timestamp_to_datetime(end_of_day_timestamp))


from datetime import datetime, timedelta
import pytz

def get_start_and_end_of_day(unix_timestamp, timezone='UTC'):
    # Convert Unix timestamp to datetime in UTC
    dt_utc = datetime.utcfromtimestamp(unix_timestamp)

    # Convert to the specified timezone
    tz = pytz.timezone(timezone)
    dt_local = tz.localize(dt_utc)

    # Start of the day
    start_of_day = dt_local.replace(hour=0, minute=0, second=0, microsecond=0)

    # End of the day
    end_of_day = start_of_day + timedelta(days=1) - timedelta(microseconds=1)

    return start_of_day, end_of_day

# Example usage:
timestamp = 1703026799  # Replace with your Unix timestamp
start, end = get_start_and_end_of_day(timestamp, timezone='Europe/Vienna')
print(f"Start of the day: {start}, End of the day: {end}")

# Convert Unix timestamp to datetime in UTC
dt_utc = datetime.utcfromtimestamp(unix_timestamp)

# Convert to the specified timezone
tz = pytz.timezone(timezone)
dt_local = tz.localize(dt_utc)

# Start of the day
start_of_day = dt_local.replace(hour=0, minute=0, second=0, microsecond=0)

# End of the day
end_of_day = start_of_day + timedelta(days=2) - timedelta(microseconds=1)


In [ ]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)

from datetime import datetime, timedelta

global conn
global cursor 


from datetime import datetime
import pytz

def get_start_and_end_of_day(timestamp, timezone='CET'):
    # Convert timestamp to datetime object
    dt_object = datetime.utcfromtimestamp(timestamp)

    # Set the timezone
    dt_object_utc = pytz.timezone('CET').localize(dt_object)
    dt_object_local = dt_object_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day
    start_of_day = dt_object_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

    return start_of_day, end_of_day

# Example usage with a timestamp
timestamp = 1703021870732 / 1000  # Assuming the input timestamp is in milliseconds
timezone = 'Europe/Vienna'  # Replace with your desired timezone

start_of_day, end_of_day = get_start_and_end_of_day(timestamp, timezone)

print(f"Start of the day: {start_of_day}")
print(f"End of the day: {end_of_day}")




In [ ]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)

from datetime import datetime, timedelta

global conn
global cursor 


from datetime import datetime
import pytz


def get_start_and_end_of_day(timestamp, timezone='CET'):
    # Convert timestamp to datetime object
    dt_object = datetime.utcfromtimestamp(timestamp)

    # Set the timezone
    dt_object_utc = pytz.timezone('CET').localize(dt_object)
    dt_object_local = dt_object_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day
    start_of_day = dt_object_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

    print(f"Awattar Before Start of the day: {start_of_day}")
    print(f"Awattar Befoe End of the day: {end_of_day}")

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp = int(start_of_day.timestamp()) * 1000
    end_timestamp = int(end_of_day.timestamp()) * 1000

    return start_timestamp, end_timestamp

# Example usage with a timestamp
timestamp = 1703021870732 / 1000  # Assuming the input timestamp is in milliseconds
timezone = 'Europe/Vienna'  # Replace with your desired timezone

start_of_day, end_of_day = get_start_and_end_of_day(timestamp, timezone)

print(f"Awattar After Start of the day: {start_of_day}")
print(f"Awattar After End of the day: {end_of_day}")



In [ ]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)

from datetime import datetime, timedelta

global conn
global cursor 


from datetime import datetime
import pytz


def get_start_and_end_of_day(timezone='CET'):
    timestamp = time.time() * 1000  # Current timestamp in milliseconds

    # Convert timestamp to datetime object
    dt_object = datetime.utcfromtimestamp(timestamp / 1000)

    # Set the timezone to UTC
    dt_object_utc = pytz.timezone('UTC').localize(dt_object)

    # Convert UTC to the specified local time zone
    dt_object_local = dt_object_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day
    start_of_day = dt_object_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Get the start of the next day
    start_of_next_day = start_of_day + timedelta(days=1)

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp = int(start_of_day.timestamp()) * 1000
    end_timestamp = int(start_of_next_day.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    print(f"Start of the day: {start_of_day}")
    print(f"End of the day (including next start): {end_of_day}")

    return start_timestamp, end_timestamp


def AWATTAR_FUTURE_PRICE():
    awattar_json_url = "https://api.awattar.at/v1/marketdata?start={}&end={}"
   
    timezone = 'Europe/Vienna'  # Replace with your desired timezone

    start_of_day, end_of_day = get_start_and_end_of_day(timezone)

    print(f"Awattar After Start of the day: {start_of_day}")
    print(f"Awattar After End of the day: {end_of_day}")

    # Get Awattar Data
    json_url = awattar_json_url.format(start_of_day, end_of_day)
    
    awattar_json_response = requests.get(json_url).json()
    awattar_json_response = pd.json_normalize(awattar_json_response['data'])
    awattar_json_response['start_timestamp'] = pd.to_datetime(awattar_json_response['start_timestamp'], unit='ms') + pd.Timedelta(hours=1)
    awattar_json_response['end_timestamp'] = pd.to_datetime(awattar_json_response['end_timestamp'], unit='ms') + pd.Timedelta(hours=1)
    print(awattar_json_response)

AWATTAR_FUTURE_PRICE()





In [ ]:
import sys 
sys.path.append('/home/pi/smart_plug/')
import os
import pandas as pd
import requests
from datetime import datetime, timedelta
from main_services.common_utils import common_utils 
import pytz
import time 

def get_start_and_end_of_day(timezone='CET'):
    timestamp = time.time() * 1000  # Current timestamp in milliseconds

    # Convert timestamp to datetime object
    dt_object = datetime.utcfromtimestamp(timestamp / 1000)

    # Set the timezone to UTC
    dt_object_utc = pytz.timezone('UTC').localize(dt_object)

    # Convert UTC to the specified local time zone
    dt_object_local = dt_object_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day
    start_of_day = dt_object_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Get the start of the next day
    start_of_next_day = start_of_day + timedelta(days=1)

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp = int(start_of_day.timestamp()) * 1000
    end_timestamp = int(start_of_next_day.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    print(f"Start of the day: {start_of_day}")
    print(f"End of the day (including next start): {end_of_day}")

    return start_timestamp, end_timestamp

# Example usage:
start_timestamp, end_timestamp = get_start_and_end_of_day('CET')
print(f"Start Timestamp: {start_timestamp}")
print(f"End Timestamp: {end_timestamp}")



In [ ]:
import sys 
sys.path.append('/home/pi/smart_plug/')
import os
import pandas as pd
import requests
from datetime import datetime, timedelta
from main_services.common_utils import common_utils 
import pytz
import time 


def get_start_and_end_of_day2(timezone='CET'):

    current_timestamp = time.time() * 1000

    timestamp = current_timestamp - (24 * 60 * 60 * 1000)

    # Convert timestamp to datetime object
    dt_object = datetime.utcfromtimestamp(timestamp / 1000)

    # Set the timezone to UTC
    dt_object_utc = pytz.timezone('UTC').localize(dt_object)

    # Convert UTC to the specified local time zone
    dt_object_local = dt_object_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day
    start_of_day = dt_object_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Get the start of the next day
    start_of_next_day = start_of_day + timedelta(days=1)

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp = int(start_of_day.timestamp()) * 1000
    end_timestamp = int(start_of_next_day.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    
    print(f"Start of the day: {start_of_day}")
    print(f"End of the day: {end_of_day}")

    return start_timestamp, end_timestamp


get_start_and_end_of_day2()


In [ ]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt
import pytz
from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

def turn_on_turn_off(relayNumber):
        # check whether the relay 1 or 2 is on Auto mode then turn on and turn off automatically
        db = DatabaseManager()
        results = db.read_relaymode_temp(relayNumber) # setting relay number
                
        current_datetime = datetime.now() 
        current_datetime = current_datetime.strftime('%Y-%m-%dT%H:00:00')

        buffer_datetime = datetime.now() + timedelta(hours=1)
        buffer_datetime = buffer_datetime.strftime('%Y-%m-%dT%H:00:00')

        print(current_datetime)
        print(buffer_datetime)

        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
        query = f"SELECT * from automaterelay where relaynumber={relayNumber} and start_timestamp BETWEEN '{current_datetime}' and '{buffer_datetime}'"
       
       # query = f"SELECT * from automaterelay where relaynumber={relayNumber} and start_timestamp BETWEEN '2023-12-07T22:00:00' and '2023-12-07T23:00:00'"
        print(query)
        
        new_dataframe = pd.read_sql_query(query, conn)

        print(new_dataframe)


        if len(results) > 0: # check whether relay is on Auto Mode
                # check if new_dataframe is empty then turn off the relay
                if(new_dataframe.empty):
                     # Turn OFF
                    print(f"Relay {relayNumber} turned off in AutoMode")

                current_time = datetime.now()
                # Check if current time is within any interval
                for index, row in new_dataframe.iterrows():
                    startdatetime = pd.to_datetime(row['start_timestamp'])
                    enddatetime = pd.to_datetime(row['end_timestamp'])
                    print("startdatetime", startdatetime)
                    print("enddatetime", enddatetime)
                    print("current_time", current_time)

                    if startdatetime <= current_time <= enddatetime:
                        # Turn On
                        print(f"Relay {relayNumber} turned on in AutoMode")
                        break
                    else:
                        # Turn OFF
                        print(f"Relay {relayNumber} turned off in AutoMode")


turn_on_turn_off(2)

In [ ]:
from datetime import datetime, timedelta
import pytz

# Replace with your desired timezone
timezone = 'Europe/Vienna'

# Set the timezone
tz = pytz.timezone(timezone)

# Get today's date with the specified timezone
today_date = datetime.now(tz).strftime("%Y-%m-%d")

# Calculate 7 days before today with the specified timezone
seven_days_before = datetime.now(tz) - timedelta(days=7)
seven_days_before_date = seven_days_before.strftime("%Y-%m-%d")

# Print the dates for verification
print("7 Days Before:", seven_days_before_date)
print("Today's Date:", today_date)

# Now you can use these dates in your URL
url = f"https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week?meterId=&startDate={seven_days_before_date}&endDate={today_date}"
print("Generated URL:", url)


In [ ]:
from datetime import datetime, timedelta

# Get today's date
today_date = datetime.now().strftime("%Y-%m-%d")

# Calculate 7 days before today
seven_days_before = datetime.now() - timedelta(days=7)
seven_days_before_date = seven_days_before.strftime("%Y-%m-%d")

# Print the dates for verification
print("7 Days Before:", seven_days_before_date)
print("Today's Date:", today_date)

# Now you can use these dates in your URL
url = f"https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week?meterId=&startDate={seven_days_before_date}&endDate={today_date}"
print("Generated URL:", url)


In [ ]:
from datetime import datetime, timedelta
import pytz

# Replace with your desired timezone
timezone = 'Europe/Vienna'

# Get the timezone object
desired_timezone = pytz.timezone(timezone)

# Get the current date and time in the desired timezone
current_datetime = datetime.now(desired_timezone)

# Calculate the date 7 days ago
seven_days_ago = current_datetime - timedelta(days=7)

# Set the time to the start of the day
start_of_day = datetime(current_datetime.year, current_datetime.month, current_datetime.day, 0, 0, 0, 0, desired_timezone)
start_of_day_seven_days_ago = datetime(seven_days_ago.year, seven_days_ago.month, seven_days_ago.day, 0, 0, 0, 0, desired_timezone)

# Set the time to the end of the day
end_of_day = datetime(current_datetime.year, current_datetime.month, current_datetime.day, 23, 59, 59, 999000, desired_timezone)
end_of_day_seven_days_ago = datetime(seven_days_ago.year, seven_days_ago.month, seven_days_ago.day, 23, 59, 59, 999000, desired_timezone)

# Convert the datetime objects to timestamps
timestamp_start_of_day_seven_days_ago = int(start_of_day_seven_days_ago.timestamp() * 1000)
timestamp_end_of_day_seven_days_ago = int(end_of_day_seven_days_ago.timestamp() * 1000)
timestamp_start_of_day_current_datetime = int(start_of_day.timestamp() * 1000)
timestamp_end_of_day_current_datetime = int(end_of_day.timestamp() * 1000)

print("From Start of Day Timestamp:", timestamp_start_of_day_seven_days_ago)
print("To End of Day Timestamp:", timestamp_end_of_day_current_datetime)


In [4]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt
import pytz
from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)


db = DatabaseManager()
result = db.check_automode_table()
print(result[0])



Current working directory: /home/pi/smart_plug/python_files
init called
2.6.0
0
